In [88]:
import pandas as pd
df_premier = pd.read_csv('premier_lgb_ready.csv')
df_premier['league'] = 0

In [89]:
df_seriea = pd.read_csv('serie_a_lgb_ready.csv')
df_seriea['league'] = 1 # poi metti in one hot

In [90]:
list(set(df_premier.columns)) == list(set(df_seriea.columns)) 

True

In [91]:
tmp = pd.concat([df_premier,df_seriea]).sort_values(by=['league_year','match_year','match_month','match_day','match_round']).reset_index(drop=True)

In [92]:
final_df = tmp.drop(['match_date','home_team','away_team','home_goals','away_goals'], axis=1)
final_df = final_df.sort_values(by = ['league_year', 'match_round']).reset_index(drop=True)
final_df = final_df.fillna(0)
final_df.head()

,10bet_1,10bet_2,10bet_X,10bet_opening_1,10bet_opening_2,10bet_opening_X,188bet_1,188bet_2,188bet_X,188bet_opening_1,...,home_team_points,home_team_goal_difference,home_team_goal_fatti,home_team_goal_subiti,away_team_rank,away_team_points,away_team_goal_difference,away_team_goal_fatti,away_team_goal_subiti,league
0,1.12,20.70,7.96,1.15,21.60,7.63,1.18,26.00,7.50,1.22,...,0,0,0,0,0,0,0,0,0,0
1,1.73,5.14,3.49,1.66,5.09,3.45,1.77,5.30,3.60,1.81,...,0,0,0,0,0,0,0,0,0,0
2,3.15,2.31,3.27,3.25,2.25,3.22,3.10,2.36,3.40,3.15,...,0,0,0,0,0,0,0,0,0,0
3,2.23,3.29,3.26,2.27,3.34,3.22,2.27,3.40,3.35,2.33,...,0,0,0,0,0,0,0,0,0,0
4,2.87,2.61,3.12,2.96,2.57,3.08,2.90,2.60,3.30,2.95,...,0,0,0,0,0,0,0,0,0,0


In [93]:
import lightgbm as lgb

from sklearn.model_selection import train_test_split

X = final_df.drop(['label'],axis=1)
y = final_df.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15,shuffle=False)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, shuffle=False)

In [94]:
from skopt.space import Real, Integer
from skopt.utils import use_named_args
space  = [Integer(1, 15, name='max_depth'),
          Real(10**-4, 10**0, "log-uniform", name='learning_rate'),
          Integer(50, 1300, name='n_estimators'),
        Integer(3,50,name='num_leaves'),
          Integer(5,50, name=' min_child_samples')
]

@use_named_args(space)
def objective(**params):
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)
    predicted_y = model.predict(X_test)
    correct = 0
    for ind,true,predicted in zip(y_test.index, y_test, predicted_y):
        if true == predicted:
            correct+=1
    return -(correct*100 / y_test.shape[0])

from skopt import forest_minimize
import numpy as np
res_gp = forest_minimize(objective, space, n_calls=10, random_state=42, verbose=False)

print("Best score=%.4f" % res_gp.fun)
print("""Best parameters:
- max_depth=%d
- learning_rate=%.6f
- n_estimators=%d
- num_leaver=%d
-  min_child_samples=%d""" 
      % (res_gp.x[0], res_gp.x[1], 
                            res_gp.x[2], res_gp.x[3], 
                            res_gp.x[4]))

Best score=-57.7193
Best parameters:
- max_depth=1
- learning_rate=0.012563
- n_estimators=560
- num_leaver=44
-  min_child_samples=32


In [95]:
model = lgb.LGBMClassifier(max_depth=1,learning_rate=0.015265, n_estimators=251, num_leaves=10,min_child_samples=17)
model.fit(X_train, y_train)
predicted_y = model.predict(X_val)
correct = 0
for ind,true,predicted in zip(y_val.index, y_val, predicted_y):
    if true == predicted:
        correct+=1
correct*100 / y_val.shape[0]

60.47471620227038

In [96]:
model = lgb.LGBMClassifier(max_depth=1,learning_rate=0.015265, n_estimators=251, num_leaves=10,min_child_samples=17)
model.fit(X_train, y_train)
predicted_prob = model.predict_proba(X_val)
predicted_prob

array([[0.30915925, 0.4427118 , 0.24812895],
       [0.25777776, 0.5353317 , 0.20689055],
       [0.28761286, 0.32685745, 0.38552969],
       ...,
       [0.2990504 , 0.41716869, 0.28378091],
       [0.30621931, 0.40319693, 0.29058377],
       [0.16313351, 0.22458554, 0.61228095]])

In [97]:
correct = 0
greater = 0
for ind,true,predicted in zip(y_val.index, y_val, predicted_prob):
    if max(predicted)>0.6:
        greater+=1
        if true == predicted.argmax():
            #print(f'probabilities: {predicted} : true: {true} max{predicted.argmax()} OK')
            correct+=1
        else:
            #print(f'probabilities: {predicted} : true: {true} max{predicted.argmax()} WRONG')
            pass
correct*100 / greater

78.43866171003718

In [98]:
# baseline che usa solo le quote
result = []
for p1,pX,p2 in zip(X_val['10bet_1'], X_val['10bet_X'], X_val['10bet_2']):
    if min([p1,pX,p2]) == p1: result.append(1)
    elif min([p1,pX,p2]) == pX: result.append(0)
    else: result.append(2)

correct = 0
for ind,true,predicted in zip(y_val.index, y_val, result):
    if true == predicted:
        correct+=1
correct*100 / y_val.shape[0]

60.577915376676984

In [99]:
list(predicted_y!=result).count(True) / len(predicted_y)

0.0196078431372549

# Proviamo ensemble

In [100]:
list(final_df.columns)

['10bet_1',
 '10bet_2',
 '10bet_X',
 '10bet_opening_1',
 '10bet_opening_2',
 '10bet_opening_X',
 '188bet_1',
 '188bet_2',
 '188bet_X',
 '188bet_opening_1',
 '188bet_opening_2',
 '188bet_opening_X',
 '1xbet_1',
 '1xbet_2',
 '1xbet_X',
 '1xbet_opening_1',
 '1xbet_opening_2',
 '1xbet_opening_X',
 '888sport_1',
 '888sport_2',
 '888sport_X',
 '888sport_opening_1',
 '888sport_opening_2',
 '888sport_opening_X',
 'bet-at-home_1',
 'bet-at-home_2',
 'bet-at-home_X',
 'bet-at-home_opening_1',
 'bet-at-home_opening_2',
 'bet-at-home_opening_X',
 'bet365_1',
 'bet365_2',
 'bet365_X',
 'bet365_opening_1',
 'bet365_opening_2',
 'bet365_opening_X',
 'betclic_1',
 'betclic_2',
 'betclic_X',
 'betclic_opening_1',
 'betclic_opening_2',
 'betclic_opening_X',
 'betfair exchange_1',
 'betfair exchange_2',
 'betfair exchange_X',
 'betfair exchange_opening_1',
 'betfair exchange_opening_2',
 'betfair exchange_opening_X',
 'betfair_1',
 'betfair_2',
 'betfair_X',
 'betfair_opening_1',
 'betfair_opening_2',
 '

In [101]:
model_1_col = [x for x in list(final_df.columns) if x.endswith('1') or x.endswith('2') or x.endswith('X')] + ['label']
model_1_col

['10bet_1',
 '10bet_2',
 '10bet_X',
 '10bet_opening_1',
 '10bet_opening_2',
 '10bet_opening_X',
 '188bet_1',
 '188bet_2',
 '188bet_X',
 '188bet_opening_1',
 '188bet_opening_2',
 '188bet_opening_X',
 '1xbet_1',
 '1xbet_2',
 '1xbet_X',
 '1xbet_opening_1',
 '1xbet_opening_2',
 '1xbet_opening_X',
 '888sport_1',
 '888sport_2',
 '888sport_X',
 '888sport_opening_1',
 '888sport_opening_2',
 '888sport_opening_X',
 'bet-at-home_1',
 'bet-at-home_2',
 'bet-at-home_X',
 'bet-at-home_opening_1',
 'bet-at-home_opening_2',
 'bet-at-home_opening_X',
 'bet365_1',
 'bet365_2',
 'bet365_X',
 'bet365_opening_1',
 'bet365_opening_2',
 'bet365_opening_X',
 'betclic_1',
 'betclic_2',
 'betclic_X',
 'betclic_opening_1',
 'betclic_opening_2',
 'betclic_opening_X',
 'betfair exchange_1',
 'betfair exchange_2',
 'betfair exchange_X',
 'betfair exchange_opening_1',
 'betfair exchange_opening_2',
 'betfair exchange_opening_X',
 'betfair_1',
 'betfair_2',
 'betfair_X',
 'betfair_opening_1',
 'betfair_opening_2',
 '

In [102]:
model_2_col = [x for x in list(final_df.columns) if x not in model_1_col] + ['label']
model_2_col

['league_year',
 'match_hour',
 'match_round',
 'match_day',
 'match_month',
 'match_year',
 'ht_win_pred_transfer',
 'draw_pred_transfer',
 'at_win_pred_transfer',
 'home_team_rank',
 'home_team_points',
 'home_team_goal_difference',
 'home_team_goal_fatti',
 'home_team_goal_subiti',
 'away_team_rank',
 'away_team_points',
 'away_team_goal_difference',
 'away_team_goal_fatti',
 'away_team_goal_subiti',
 'league',
 'label']

In [103]:
model_1_df = final_df[model_1_col]
X_1 = model_1_df.drop(['label'],axis=1)
y_1 = model_1_df.label

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.15,shuffle=False)

X_train_1, X_val_1, y_train_1, y_val_1 = train_test_split(X_train_1, y_train_1, test_size=0.15, shuffle=False)

model_2_df = final_df[model_2_col]
X_2 = model_2_df.drop(['label'],axis=1)
y_2 = model_2_df.label

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.15,shuffle=False)

X_train_2, X_val_2, y_train_2, y_val_2 = train_test_split(X_train_2, y_train_2, test_size=0.15, shuffle=False)

In [104]:
#chekc
(X_train_1.shape[0] ==  X_train_2.shape[0]) and (X_test_1.shape[0]==X_test_2.shape[0]) and (y_train_1.shape[0]==y_train_2.shape[0]) and (y_test_1.shape[0]==y_test_2.shape[0]) and (y_train_1.shape[0]==y_train_2.shape[0]) and(y_val_1.shape[0]==y_val_2.shape[0])

True

# Estraggo scores per il test

In [105]:
from skopt.space import Real, Integer
from skopt.utils import use_named_args
space  = [Integer(1, 15, name='max_depth'),
          Real(10**-4, 10**0, "log-uniform", name='learning_rate'),
          Integer(50, 1300, name='n_estimators'),
        Integer(3,50,name='num_leaves'),
          Integer(5,50, name=' min_child_samples')
]

In [110]:
@use_named_args(space)
def objective_1(**params):
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train_1, y_train_1)
    predicted_y = model.predict(X_test_1)
    correct = 0
    for ind,true,predicted in zip(y_test_1.index, y_test_1, predicted_y):
        if true == predicted:
            correct+=1
    return -(correct*100 / y_test_1.shape[0])

from skopt import forest_minimize
import numpy as np
res_gp = forest_minimize(objective_1, space, n_calls=100, random_state=42, verbose=False)

print("Best score=%.4f" % res_gp.fun)
print("""Best parameters:
- max_depth=%d
- learning_rate=%.6f
- n_estimators=%d
- num_leaver=%d
-  min_child_samples=%d""" 
      % (res_gp.x[0], res_gp.x[1], 
                            res_gp.x[2], res_gp.x[3], 
                            res_gp.x[4]))

Best score=-58.1579
Best parameters:
- max_depth=1
- learning_rate=0.011169
- n_estimators=375
- num_leaver=40
-  min_child_samples=50


In [111]:
model_1 = lgb.LGBMClassifier(max_depth=1,learning_rate=0.011169, n_estimators=375, num_leaves=40,min_child_samples=50)
model_1.fit(X_train_1, y_train_1)
predicted_y_1 = model_1.predict(X_test_1)
correct = 0
for ind,true,predicted in zip(y_test_1.index, y_test_1, predicted_y_1):
    if true == predicted:
        correct+=1
correct*100 / y_test_1.shape[0]

58.1578947368421

In [112]:
@use_named_args(space)
def objective_2(**params):
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train_2, y_train_2)
    predicted_y = model.predict(X_test_2)
    correct = 0
    for ind,true,predicted in zip(y_test_2.index, y_test_2, predicted_y):
        if true == predicted:
            correct+=1
    return -(correct*100 / y_test_2.shape[0])

from skopt import forest_minimize
import numpy as np
res_gp = forest_minimize(objective_2, space, n_calls=100, random_state=17, verbose=False)

print("Best score=%.4f" % res_gp.fun)
print("""Best parameters:
- max_depth=%d
- learning_rate=%.6f
- n_estimators=%d
- num_leaver=%d
-  min_child_samples=%d""" 
      % (res_gp.x[0], res_gp.x[1], 
                            res_gp.x[2], res_gp.x[3], 
                            res_gp.x[4]))

Best score=-56.4912
Best parameters:
- max_depth=4
- learning_rate=0.001607
- n_estimators=1020
- num_leaver=42
-  min_child_samples=36


In [113]:
model_2 = lgb.LGBMClassifier(max_depth=4,learning_rate=0.001607, n_estimators=1020, num_leaves=42,min_child_samples=36)
model_2.fit(X_train_2, y_train_2)
predicted_y_2 = model_2.predict(X_test_2)
correct = 0
for ind,true,predicted in zip(y_test_2.index, y_test_2, predicted_y_2):
    if true == predicted:
        correct+=1
correct*100 / y_test_1.shape[0]

56.14035087719298

In [114]:
predicted_y_1 = model_1.predict(X_test_1,raw_score=True)
predicted_y_2 = model_2.predict(X_test_2, raw_score=True)

In [115]:
stacked_df = pd.DataFrame(columns=['m1X','m11','m12','m2X','m21','m22'])
stacked_df['m1X'] = [x[0] for x in predicted_y_1]
stacked_df['m11'] = [x[1] for x in predicted_y_1]
stacked_df['m12'] = [x[2] for x in predicted_y_1]
stacked_df['m2X'] = [x[0] for x in predicted_y_2]
stacked_df['m21'] = [x[1] for x in predicted_y_2]
stacked_df['m22'] = [x[2] for x in predicted_y_2]

In [116]:
stacked_df.head()

,m1X,m11,m12,m2X,m21,m22
0,-1.538554,-0.220132,-2.033261,-1.367497,-0.479922,-1.850166
1,-1.261715,-1.065773,-1.317323,-1.218627,-0.918286,-1.302318
2,-2.074611,-0.116453,-2.047590,-1.420362,-0.260274,-1.980360
3,-1.261715,-0.983686,-1.317323,-1.247342,-0.812576,-1.437067
4,-2.074611,-0.116453,-2.033261,-1.434163,-0.063633,-1.862931


# Estraggo scores per validation

In [117]:
model_1 = lgb.LGBMClassifier(max_depth=1,learning_rate=0.011169, n_estimators=375, num_leaves=40,min_child_samples=50)
model_1.fit(X_train_1, y_train_1)
predicted_y_1 = model_1.predict(X_val_1, raw_score=True)

model_2 = lgb.LGBMClassifier(max_depth=4,learning_rate=0.001607, n_estimators=1020, num_leaves=42,min_child_samples=36)
model_2.fit(X_train_2, y_train_2)
predicted_y_2 = model_2.predict(X_val_2, raw_score=True)

In [124]:
val_stacked_df = pd.DataFrame(columns=['m1X','m11','m12','m2X','m21','m22'])
val_stacked_df['m1X'] = [x[0] for x in predicted_y_1]
val_stacked_df['m11'] = [x[1] for x in predicted_y_1]
val_stacked_df['m12'] = [x[2] for x in predicted_y_1]
val_stacked_df['m2X'] = [x[0] for x in predicted_y_2]
val_stacked_df['m21'] = [x[1] for x in predicted_y_2]
val_stacked_df['m22'] = [x[2] for x in predicted_y_2]
val_stacked_df.head()

,m1X,m11,m12,m2X,m21,m22
0,-1.261715,-0.897097,-1.484787,-1.262047,-0.859429,-1.234364
1,-1.261715,-0.519437,-1.484787,-1.277050,-0.686362,-1.489617
2,-1.261715,-1.144647,-0.970399,-1.252927,-1.092742,-1.109456
3,-1.261715,-1.249153,-0.811695,-1.252927,-0.920320,-1.266436
4,-1.918311,-1.363368,-0.391039,-1.312160,-1.406569,-0.430857


# Ora prediciamo per la validation

In [136]:
ensemble = lgb.LGBMClassifier(max_depth=1,learning_rate=0.017, n_estimators=300, num_leaves=50,min_child_samples=30)
ensemble.fit(stacked_df, list(y_test))

predicted_y = ensemble.predict(val_stacked_df)
correct = 0
for ind,true,predicted in zip(y_val.index, y_val, predicted_y):
    if true == predicted:
        correct+=1
correct*100 / y_val.shape[0]

60.68111455108359